<a href="https://colab.research.google.com/github/HimalKarkal/netball-analysis/blob/master/Glicko_Rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math

class GlickoRating:
    def __init__(self, r=1500, rd=350):
        self.r = r
        self.rd = rd

q = math.log(10) / 400

def g(rd):
  return 1 / math.sqrt(1 + (3 * q**2 * rd**2) / math.pi**2)

def E(r, ro, rdo):
    return 1 / (1 + 10**(-g(rdo) * (r - ro) / 400))

def o(margin, positiveStat = True):
  if positiveStat == True:
    return 1 / (1 + math.exp(-margin))
  else:
    return 1 / (1 + math.exp(margin))

def d2(r, ro, rdo):
  return (q**2 * g(rdo)**2 * E(r, ro, rdo) * (1 - E(r, ro, rdo)) + 1e-9)**-1 #Added epsilon to prevent error.

def rate(ratingP, ratingO, margin, positiveStat=True):
  r = ratingP.r
  rd = ratingP.rd
  ro = ratingO.r
  rdo = ratingO.rd
  e = E(r, ro, rdo)

  if positiveStat == True:
    outcome = o(margin, positiveStat = True)
  else:
    outcome = o(margin, positiveStat = False)

  r_updated = r + (q / ((1 / rd**2) + 1 / d2(r, ro, rdo))) * g(rdo) * (outcome - e)
  rd_updated = math.sqrt(1 / ((1 / rd**2) + 1 / d2(r, ro, rdo)))

  return GlickoRating(r_updated, rd_updated)

#Applying Glicko to SSN 2020-23

In [3]:
# Cloning the data from GitHub

! git clone 'https://github.com/HimalKarkal/netball-analysis.git'

Cloning into 'netball-analysis'...
remote: Enumerating objects: 26868, done.
remote: Counting objects: 100% (3096/3096), done.
remote: Compressing objects: 100% (2125/2125), done.
remote: Total 26868 (delta 879), reused 3077 (delta 871), pack-reused 23772
Receiving objects: 100% (26868/26868), 45.30 MiB | 21.68 MiB/s, done.
Resolving deltas: 100% (13066/13066), done.
Updating files: 100% (14889/14889), done.


In [4]:
# Importing necessary modules

import glob
import numpy as np
import pandas as pd
from scipy import stats

In [5]:
# Weights for positions

dict_weights = {
'GS':{
    "goal1": 0.35,
    "goalMisses": 0.2,
    "goal2": 0.15,
    "generalPlayTurnovers": 0.1,
    "rebounds": 0.07,
    "blocked": 0.05,
    "penalties": 0.05,
    "feedWithAttempt": 0.01,
    "feedWithoutAttempt": 0.01,
    "pickups": 0.01
},

'GA':{
    "goal1": 0.25,
    "goalMisses": 0.15,
    "goal2": 0.1,
    "feedWithAttempt": 0.1,
    "generalPlayTurnovers": 0.1,
    "centrePassReceives": 0.07,
    "feedWithoutAttempt": 0.07,
    "blocked": 0.05,
    "rebounds": 0.05,
    "penalties": 0.04,
    "gain": 0.01,
    "pickups": 0.01
},

'WA':{
    "centrePassReceives": 0.3,
    "feedWithAttempt": 0.25,
    "generalPlayTurnovers": 0.15,
    "feedWithoutAttempt": 0.1,
    "penalties": 0.07,
    "gain": 0.05,
    "pickups": 0.05,
    "deflections": 0.03
},

'C': {
    "generalPlayTurnovers": 0.35,
    "feedWithAttempt": 0.3,
    "feedWithoutAttempt": 0.15,
    "penalties": 0.07,
    "pickups": 0.07,
    "gain": 0.05,
    "deflections": 0.01
},

'WD': {
    "deflections": 0.3,
    "penalties": 0.25,
    "gain": 0.2,
    "generalPlayTurnovers": 0.1,
    "centrePassReceives": 0.07,
    "pickups": 0.05,
    "feedWithAttempt": 0.02,
    "feedWithoutAttempt": 0.01
},

'GD': {
    "deflections": 0.35,
    "penalties": 0.2,
    "gain": 0.15,
    "blocks": 0.1,
    "rebounds": 0.1,
    "generalPlayTurnovers": 0.05,
    "centrePassReceives": 0.03,
    "pickups": 0.02
},

'GK': {
    "deflections": 0.35,
    "penalties": 0.25,
    "gain": 0.15,
    "blocks": 0.1,
    "rebounds": 0.1,
    "generalPlayTurnovers": 0.03,
    "pickups": 0.02
}
}

In [45]:
# Final Code

df_team_ratings = pd.DataFrame()

dict_player_ratings = {'attempt1':{},
                        'attempt2':{},
                        'attempt_from_zone1':{},
                        'attempt_from_zone2':{},
                        'badHands':{},
                        'badPasses':{},
                        'blocked':{},
                        'blocks':{},
                        'breaks':{},
                        'centrePassReceives':{},
                        'centrePassToGoalPerc':{},
                        'centrePassToGoalPerc':{},
                        'contactPenalties':{},
                        'deflectionPossessionGain':{},
                        'deflectionWithGain':{},
                        'deflectionWithNoGain':{},
                        'deflections':{},
                        'disposals':{},
                        'feedWithAttempt':{},
                        'feedWithoutAttempt':{},
                        'feeds':{},
                        'gain':{},
                        'gainToGoalPerc':{},
                        'generalPlayTurnovers':{},
                        'goal1':{},
                        'goal2':{},
                        'goalAssists':{},
                        'goalAttempts':{},
                        'goalMisses':{},
                        'goal_from_zone1':{},
                        'goal_from_zone2':{},
                        'goals':{},
                        'interceptPassThrown':{},
                        'intercepts':{},
                        'obstructionPenalties':{},
                        'offsides':{},
                        'passes':{},
                        'penalties':{},
                        'pickups':{},
                        'rebounds':{}}

dict_relative_league_ratings = {'attempt1':{},
                        'attempt2':{},
                        'attempt_from_zone1':{},
                        'attempt_from_zone2':{},
                        'badHands':{},
                        'badPasses':{},
                        'blocked':{},
                        'blocks':{},
                        'breaks':{},
                        'centrePassReceives':{},
                        'centrePassToGoalPerc':{},
                        'centrePassToGoalPerc':{},
                        'contactPenalties':{},
                        'deflectionPossessionGain':{},
                        'deflectionWithGain':{},
                        'deflectionWithNoGain':{},
                        'deflections':{},
                        'disposals':{},
                        'feedWithAttempt':{},
                        'feedWithoutAttempt':{},
                        'feeds':{},
                        'gain':{},
                        'gainToGoalPerc':{},
                        'generalPlayTurnovers':{},
                        'goal1':{},
                        'goal2':{},
                        'goalAssists':{},
                        'goalAttempts':{},
                        'goalMisses':{},
                        'goal_from_zone1':{},
                        'goal_from_zone2':{},
                        'goals':{},
                        'interceptPassThrown':{},
                        'intercepts':{},
                        'obstructionPenalties':{},
                        'offsides':{},
                        'passes':{},
                        'penalties':{},
                        'pickups':{},
                        'rebounds':{}}
dict_player_stats = {'attempt1':{},
                        'attempt2':{},
                        'attempt_from_zone1':{},
                        'attempt_from_zone2':{},
                        'badHands':{},
                        'badPasses':{},
                        'blocked':{},
                        'blocks':{},
                        'breaks':{},
                        'centrePassReceives':{},
                        'centrePassToGoalPerc':{},
                        'centrePassToGoalPerc':{},
                        'contactPenalties':{},
                        'deflectionPossessionGain':{},
                        'deflectionWithGain':{},
                        'deflectionWithNoGain':{},
                        'deflections':{},
                        'disposals':{},
                        'feedWithAttempt':{},
                        'feedWithoutAttempt':{},
                        'feeds':{},
                        'gain':{},
                        'gainToGoalPerc':{},
                        'generalPlayTurnovers':{},
                        'goal1':{},
                        'goal2':{},
                        'goalAssists':{},
                        'goalAttempts':{},
                        'goalMisses':{},
                        'goal_from_zone1':{},
                        'goal_from_zone2':{},
                        'goals':{},
                        'interceptPassThrown':{},
                        'intercepts':{},
                        'obstructionPenalties':{},
                        'offsides':{},
                        'passes':{},
                        'penalties':{},
                        'pickups':{},
                        'rebounds':{}}
dict_league_stats = {'attempt1':{},
                        'attempt2':{},
                        'attempt_from_zone1':{},
                        'attempt_from_zone2':{},
                        'badHands':{},
                        'badPasses':{},
                        'blocked':{},
                        'blocks':{},
                        'breaks':{},
                        'centrePassReceives':{},
                        'centrePassToGoalPerc':{},
                        'centrePassToGoalPerc':{},
                        'contactPenalties':{},
                        'deflectionPossessionGain':{},
                        'deflectionWithGain':{},
                        'deflectionWithNoGain':{},
                        'deflections':{},
                        'disposals':{},
                        'feedWithAttempt':{},
                        'feedWithoutAttempt':{},
                        'feeds':{},
                        'gain':{},
                        'gainToGoalPerc':{},
                        'generalPlayTurnovers':{},
                        'goal1':{},
                        'goal2':{},
                        'goalAssists':{},
                        'goalAttempts':{},
                        'goalMisses':{},
                        'goal_from_zone1':{},
                        'goal_from_zone2':{},
                        'goals':{},
                        'interceptPassThrown':{},
                        'intercepts':{},
                        'obstructionPenalties':{},
                        'offsides':{},
                        'passes':{},
                        'penalties':{},
                        'pickups':{},
                        'rebounds':{}}

dict_absolute_ratings = {}

tournament = "SSN" # Use this to change the tournament

for season in ['2020','2021','2022','2023']: # Use this to choose the seasons
  list_subs = glob.glob("/content/netball-analysis/data/matchCentre/processed/*/" + "*substitutions*" + season + "*" + tournament + "*.csv")
  list_playerStats = glob.glob("/content/netball-analysis/data/matchCentre/processed/*/" + "*playerStats*" + season + "*" + tournament + "*.csv")
  list_teamStats = glob.glob("/content/netball-analysis/data/matchCentre/processed/*/" + "*teamStats*" + season + "*" + tournament + "*.csv")

  list_subs.sort()
  list_playerStats.sort()
  list_teamStats.sort()

  dict_subs = {}
  dict_playerStats = {}
  dict_teamStats = {}

  i = 1
  while i <= 14:
    dict_subs[i] = list_subs[4 * (i-1): 4 * i]
    dict_playerStats[i] = list_playerStats[4 * (i-1): 4 * i]
    dict_teamStats[i] = list_teamStats[4 * (i-1): 4 * i]

    i += 1

  # Rating function
  '''
  Accepts: Dictionaries containing filepaths (dict_subs and dict_playerStats)
  Returns: Concatenated dataframes for each round in the season. (df_subs and df_playerStats)
  '''
  for week in range(1,15):

    df_subs = pd.DataFrame()
    df_playerStats = pd.DataFrame()
    df_teamStats = pd.DataFrame()

    for game in range(0,4):
      subs_temp = pd.read_csv(dict_subs[week][game])
      playerStats_temp = pd.read_csv(dict_playerStats[week][game])
      teamStats_temp = pd.read_csv(dict_teamStats[week][game])

      df_subs = pd.concat([df_subs,subs_temp])
      df_playerStats = pd.concat([df_playerStats,playerStats_temp])
      df_teamStats = pd.concat([df_teamStats,teamStats_temp])

    del subs_temp, playerStats_temp, teamStats_temp

    '''
    Accepts: Substitutions dataframe for each week (df_subs)
    Returns: Classification of each player to a position (df_temp['Position'])
    Comments: Players who spent the entire time as a substitutes are excluded
    '''
    dict_temp = {
    'GS':{},
    'GA':{},
    'WA': {},
    'C': {},
    'WD': {},
    'GD': {},
    'GK': {},
    'S': {}
    }

    for i, row in df_subs.iterrows():
      player = row['playerId']
      position = row['startingPos']
      duration = row['duration']

      if player in dict_temp[position]:
        dict_temp[position][player] += duration
      else:
        dict_temp[position][player] = duration

    df_timeInPosition = pd.DataFrame(dict_temp).fillna(0).reset_index()
    df_timeInPosition = df_timeInPosition.rename(columns={'index': 'playerId'})
    df_timeInPosition = df_timeInPosition.loc[df_timeInPosition['S'] != 3600]
    df_timeInPosition['position'] = df_timeInPosition.drop(columns = ['playerId', 'S']).idxmax(axis = 1)

    #Selecting appropriate columns only from df_playerStats

    df_playerStats = df_playerStats[['matchId', 'squadId', 'oppSquadId', 'playerId', 'attempt1', 'attempt2',
          'attempt_from_zone1', 'attempt_from_zone2', 'badHands', 'badPasses',
          'blocked', 'blocks', 'breaks', 'centrePassReceives',
          'centrePassToGoalPerc', 'contactPenalties', 'deflectionPossessionGain',
          'deflectionWithGain', 'deflectionWithNoGain', 'deflections',
          'feedWithAttempt', 'feeds', 'gain', 'gainToGoalPerc',
          'generalPlayTurnovers', 'goal1', 'goal2', 'goalAssists', 'goalAttempts',
          'goalMisses', 'goal_from_zone1', 'goal_from_zone2', 'goals',
          'interceptPassThrown', 'intercepts', 'minutesPlayed',
          'missedGoalTurnover', 'netPoints', 'obstructionPenalties', 'offsides',
          'penalties', 'pickups', 'points', 'possessionChanges',
          'quartersPlayed', 'rebounds', 'tossUpWin']]
    df_playerStats['feedWithoutAttempt'] = df_playerStats['feeds'] - df_playerStats['feedWithAttempt']
    df_playerStats = df_playerStats.merge(df_timeInPosition[['playerId', 'position']], on = 'playerId', how = 'left')
    df_playerStats = df_playerStats.dropna()

    #Calculating rate of scoring each statistic
    relevant_stats = ['goal1', 'goal2', 'rebounds', 'feedWithAttempt',
                        'feedWithoutAttempt', 'pickups', 'centrePassReceives',
                        'gain','deflections','blocks','blocked', 'goalMisses',
                        'generalPlayTurnovers', 'penalties']

    for statistic in relevant_stats:
      df_playerStats[statistic] = df_playerStats[statistic] / df_playerStats['minutesPlayed']

    df_playerStats = df_playerStats[['playerId','squadId','goal1', 'goal2', 'rebounds', 'feedWithAttempt',
                        'feedWithoutAttempt', 'pickups', 'centrePassReceives',
                        'gain','deflections','blocks','blocked', 'goalMisses',
                        'generalPlayTurnovers', 'penalties','minutesPlayed','position']]
    df_playerStats = df_playerStats.loc[df_playerStats['minutesPlayed']>= 10]

    del df_timeInPosition

    # Calculating Means

    df_means = df_playerStats.groupby('position').mean().round(2).reset_index()
    df_stds = df_playerStats.groupby('position').std().round(2).reset_index()

    # Rating players

    for i, player in df_playerStats.iterrows():
      position = player.loc['position']
      row_means = df_means.loc[df_means['position'] == position]
      row_stds = df_stds.loc[df_stds['position'] == position]
      player_Id = player['playerId']


      # Positive statistics


      for statistic in ['goal1', 'goal2', 'rebounds', 'feedWithAttempt',
                        'feedWithoutAttempt', 'pickups', 'centrePassReceives',
                        'gain','deflections','blocks']: # Check and add other positive statistics
        if statistic in dict_weights[position].keys():

          # Accessing player rating for statistic from dictionary or creating if non-existent

          if player_Id not in dict_player_ratings[statistic]:
            dict_player_ratings[statistic][player_Id] = [GlickoRating()]

          if player_Id not in dict_relative_league_ratings[statistic]:
            dict_relative_league_ratings[statistic][player_Id] = [GlickoRating()]

          if player_Id not in dict_player_stats[statistic]:
            dict_player_stats[statistic][player_Id] = []

          if player_Id not in dict_league_stats[statistic]:
            dict_league_stats[statistic][player_Id] = []

          player_rating = dict_player_ratings[statistic][player_Id][-1]
          league_average_rating = dict_relative_league_ratings[statistic][player_Id][-1]

          #Calculating Margin

          sd = row_stds.iloc[0][statistic]
          if sd != 0:
            margin = (player[statistic] - row_means.iloc[0][statistic])/sd
          else:
            margin = 0

          # Rating players and league for statistic

          player_rating = rate(player_rating, league_average_rating, margin, positiveStat = True)
          league_average_rating = rate(league_average_rating, player_rating, 0, positiveStat = True)

          dict_player_ratings[statistic][player_Id].append(player_rating)
          dict_relative_league_ratings[statistic][player_Id].append(league_average_rating)
          dict_player_stats[statistic][player_Id].append(player[statistic])
          dict_league_stats[statistic][player_Id].append(row_means.iloc[0][statistic])


        # Negative statistics


      for statistic in ['blocked', 'goalMisses',
                        'generalPlayTurnovers', 'penalties']: #Add other negative statistics
        if statistic in dict_weights[position].keys():

          # Accessing player rating for statistic from dictionary or creating if non-existent

          if player_Id not in dict_player_ratings[statistic]:
            dict_player_ratings[statistic][player_Id] = [GlickoRating()]

          if player_Id not in dict_relative_league_ratings[statistic]:
            dict_relative_league_ratings[statistic][player_Id] = [GlickoRating()]

          if player_Id not in dict_player_stats[statistic]:
            dict_player_stats[statistic][player_Id] = []

          if player_Id not in dict_league_stats[statistic]:
            dict_league_stats[statistic][player_Id] = []

          player_rating = dict_player_ratings[statistic][player_Id][-1]
          league_average_rating =  dict_relative_league_ratings[statistic][player_Id][-1]

          #Calculating Margin

          sd = row_stds.iloc[0][statistic]
          if sd != 0:
            margin = (player[statistic] - row_means.iloc[0][statistic]) / sd
          else:
            margin = 0

          # Rating players and league for statistic

          player_rating = rate(player_rating, league_average_rating, margin, positiveStat = False)
          league_average_rating = rate(league_average_rating, player_rating, 0, positiveStat = False)

          dict_player_ratings[statistic][player_Id].append(player_rating)
          dict_relative_league_ratings[statistic][player_Id].append(league_average_rating)
          dict_player_stats[statistic][player_Id].append(player[statistic])
          dict_league_stats[statistic][player_Id].append(row_means.iloc[0][statistic])

      #Generating absolute ratings
      if player_Id not in dict_absolute_ratings:
        dict_absolute_ratings[player_Id] = [1500]

      list_stats = dict_weights[position].keys()
      absolute_rating = 0
      for statistic in list_stats:
        absolute_rating += dict_weights[position][statistic] * dict_player_ratings[statistic][player_Id][-1].r

      dict_absolute_ratings[player_Id].append(absolute_rating)
      df_playerStats.at[i, 'glickoRating'] = absolute_rating

    # This section adds nans to a player's rating list if they have not been rated in a particular week
    # This ensures all rounds are in sync
    for player in dict_absolute_ratings:
      if player not in list(df_playerStats['playerId'].values):
        dict_absolute_ratings[player].append(np.nan)

    for player_id, ratings in dict_absolute_ratings.items():
      max_length = max(len(r) for r in dict_absolute_ratings.values())
      dict_absolute_ratings[player_id] = [np.nan] * (max_length - len(ratings)) + ratings

    # Combining team Glicko ratings to teamStats and df_team_ratings
    team_average_ratings = df_playerStats[['squadId', 'glickoRating']].groupby('squadId').mean().round(2).reset_index()
    df_teamStats = df_teamStats.merge(team_average_ratings, on = 'squadId', how = 'left')
    df_team_ratings = pd.concat(objs =[df_team_ratings, df_teamStats], axis = 0)

<ipython-input-45-ab4ea69baf6f>:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_playerStats['feedWithoutAttempt'] = df_playerStats['feeds'] - df_playerStats['feedWithAttempt']
<ipython-input-45-ab4ea69baf6f>:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_playerStats['feedWithoutAttempt'] = df_playerStats['feeds'] - df_playerStats['feedWithAttempt']


In [50]:
df_team_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 448 entries, 0 to 7
Data columns (total 62 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   matchId                   448 non-null    int64  
 1   squadId                   448 non-null    int64  
 2   oppSquadId                448 non-null    int64  
 3   attempt1                  448 non-null    float64
 4   attempt2                  448 non-null    float64
 5   attempt_from_zone1        448 non-null    float64
 6   attempt_from_zone2        448 non-null    float64
 7   badHands                  448 non-null    float64
 8   badPasses                 448 non-null    float64
 9   blocked                   448 non-null    float64
 10  blocks                    448 non-null    float64
 11  breaks                    448 non-null    float64
 12  centrePassReceives        448 non-null    float64
 13  centrePassToGoalPerc      448 non-null    float64
 14  contactPenalties 

# This is That Bit

In [ ]:
# Big complicated table with absolute and statistic ratings

# Creating a Final Table

paths = glob.glob('/content/netball-analysis/data/matchCentre/processed/*/*playerList_202*_SSN_*.csv')

player_list_dict = {}

for i in paths:
  df = pd.read_csv(i)

  for i, row in df.iterrows():
    if row['playerId'] not in player_list_dict:
      player_list_dict[row['playerId']] = row['displayName']

df_player_list = pd.DataFrame(player_list_dict.items(), columns = ['playerId', 'displayName'])

dict_final_absolute_ratings = {}

for player in dict_absolute_ratings:
  if len(dict_absolute_ratings[player]) >= 20:
    dict_final_absolute_ratings[player] = {}
    dict_final_absolute_ratings[player]['absoluteRating'] = dict_absolute_ratings[player][-1]

    for statistic in ['goal1', 'goal2', 'rebounds', 'feedWithAttempt','feedWithoutAttempt', 'pickups', 'centrePassReceives',
                        'gain','deflections','blocks','blocked', 'goalMisses',
                        'generalPlayTurnovers', 'penalties']:
      try:
        dict_final_absolute_ratings[player][statistic] = dict_player_ratings[statistic][player][-1].r
      except KeyError:
        pass

df_final_absolute_ratings = pd.DataFrame()
for player in dict_final_absolute_ratings.keys():
  df_temp = pd.DataFrame(dict_final_absolute_ratings[player], index = [0])
  df_temp['playerId'] = player
  df_final_absolute_ratings = pd.concat([df_final_absolute_ratings, df_temp])

df_final_absolute_ratings = df_final_absolute_ratings.merge(df_player_list, on = 'playerId', how = 'left')

In [ ]:
df_final_absolute_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   absoluteRating        40 non-null     float64
 1   goal1                 13 non-null     float64
 2   goal2                 13 non-null     float64
 3   rebounds              27 non-null     float64
 4   feedWithAttempt       29 non-null     float64
 5   feedWithoutAttempt    29 non-null     float64
 6   pickups               40 non-null     float64
 7   blocked               13 non-null     float64
 8   goalMisses            13 non-null     float64
 9   generalPlayTurnovers  40 non-null     float64
 10  penalties             40 non-null     float64
 11  playerId              40 non-null     int64  
 12  centrePassReceives    28 non-null     float64
 13  gain                  34 non-null     float64
 14  deflections           28 non-null     float64
 15  blocks                14 

In [ ]:
df_final_absolute_ratings.to_csv('Glicko_ratings_with_stats.csv')

In [ ]:
# Uncomplicated table to calculate team average glicko ratings

paths = glob.glob('/content/netball-analysis/data/matchCentre/processed/*/*playerList_202*_SSN_*.csv')

df_player_list = pd.DataFrame(columns = ['playerId', 'displayName', 'squadId'])

player_displayName_dict = {}
player_squadId_dict = {}

for i in paths:
  df = pd.read_csv(i)

  for i, row in df.iterrows():
    if row['playerId'] not in player_displayName_dict:
      player_displayName_dict[row['playerId']] = row['displayName']
      player_squadId_dict[row['playerId']] = row['squadId']

for player in player_displayName_dict:
  df_player_list.loc[len(df_player_list)] = [player, player_displayName_dict[player], player_squadId_dict[player]]

df_final_absolute_ratings = pd.DataFrame(columns = ['playerId', 'rating'])

for player in dict_absolute_ratings:
    df_final_absolute_ratings.loc[len(df_final_absolute_ratings)] = [player, dict_absolute_ratings[player][-1]]

df_final_absolute_ratings = df_final_absolute_ratings.merge(df_player_list, on = 'playerId', how = 'left')

In [ ]:
df_final_absolute_ratings.head()

,playerId,rating,displayName,squadId
0,80078.0,1450.990487,R.Aiken,807
1,80439.0,1531.436894,M.Proud,806
2,80574.0,1513.285967,P.Hadley,806
3,998404.0,1494.873126,S.Klau,806
4,999128.0,1617.802042,H.Housby,806


In [ ]:
df_means = df_final_absolute_ratings.drop(columns = {'displayName', 'playerId'}).groupby('squadId').mean().round(2).reset_index()

In [ ]:
df_means.to_csv('Glicko_ratings_by_squad_2022.csv')